In [1]:
!pip install lightgbm

In [2]:
import os
from data_split import *
from feature_additions import *

data_path = 'data'
events_df = pd.read_csv('data/events.csv')  # [user_id, item_id, rating, timestamp]

user_features_df = pd.read_csv(os.path.join(data_path,'user_features.csv')) # [user_id, gender, age]
item_features_df = pd.read_csv('./data/item_features.csv')

train_df, val_df, test_df = split_data_by_user(events_df)

user_features_from_train = get_user_features_from_train(train_df, item_features_df)
item_features_from_train = get_item_features_from_train(train_df, item_features_df)

# Присоединение признаков к тестовому набору данных
train_df_with_features = join_features_to_test(train_df, user_features_from_train, item_features_from_train)
test_df_with_features = join_features_to_test(test_df, user_features_from_train, item_features_from_train)
val_df_with_features = join_features_to_test(val_df, user_features_from_train, item_features_from_train)


target_col = ['rating']
X_train, y_train = train_df_with_features.drop(target_col, axis = 1), train_df_with_features[target_col]
X_test, y_test = test_df_with_features.drop(target_col, axis = 1), test_df_with_features[target_col]
X_val, y_val = val_df_with_features.drop(target_col, axis = 1), val_df_with_features[target_col]



In [3]:
X_val

,user_id,item_id,user_avg_rating,user_num_ratings,user_avg_rating_genre_0,user_avg_rating_genre_1,user_avg_rating_genre_2,user_avg_rating_genre_3,user_avg_rating_genre_4,user_avg_rating_genre_5,...,user_avg_rating_genre_11,user_avg_rating_genre_12,user_avg_rating_genre_13,user_avg_rating_genre_14,user_avg_rating_genre_15,user_avg_rating_genre_16,user_avg_rating_genre_17,num_genres,movie_avg_rating,movie_num_ratings
0,0,1809,3.992883,281,0.327402,0.217082,0.092527,0.153025,0.370107,0.067616,...,0.064057,0.042705,0.192171,0.167260,0.217082,0.060498,0.014235,2,3.548193,996.0
1,0,1124,3.992883,281,0.327402,0.217082,0.092527,0.153025,0.370107,0.067616,...,0.064057,0.042705,0.192171,0.167260,0.217082,0.060498,0.014235,2,3.488660,485.0
2,0,1792,3.992883,281,0.327402,0.217082,0.092527,0.153025,0.370107,0.067616,...,0.064057,0.042705,0.192171,0.167260,0.217082,0.060498,0.014235,1,3.671617,606.0
3,1,2993,3.666667,255,0.321569,0.125490,0.027451,0.035294,0.258824,0.121569,...,0.019608,0.035294,0.090196,0.231373,0.403922,0.050980,0.019608,5,2.904762,252.0
4,1,3151,3.666667,255,0.321569,0.125490,0.027451,0.035294,0.258824,0.121569,...,0.019608,0.035294,0.090196,0.231373,0.403922,0.050980,0.019608,2,4.255963,1090.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18115,6038,3574,3.780526,647,0.224111,0.180835,0.094281,0.188563,0.457496,0.044822,...,0.083462,0.030912,0.120556,0.157651,0.102009,0.046368,0.018547,1,2.518797,266.0
18116,6038,3246,3.780526,647,0.224111,0.180835,0.094281,0.188563,0.457496,0.044822,...,0.083462,0.030912,0.120556,0.157651,0.102009,0.046368,0.018547,2,3.150943,53.0
18117,6039,613,3.801020,196,0.301020,0.198980,0.020408,0.025510,0.244898,0.061224,...,0.010204,0.040816,0.188776,0.362245,0.183673,0.040816,0.015306,2,4.248588,354.0
18118,6039,1453,3.801020,196,0.301020,0.198980,0.020408,0.025510,0.244898,0.061224,...,0.010204,0.040816,0.188776,0.362245,0.183673,0.040816,0.015306,1,3.767007,588.0


In [4]:
import lightgbm as lgb

lgb_train = lgb.Dataset(X_train, y_train)
lgb_val = lgb.Dataset(X_val, y_val, reference=lgb_train)

# Параметры модели
params = {
    'objective': 'regression',      # Можно использовать 'lambdarank' для ранжирования
    'metric': 'rmse',               # Метрика регрессии, для ранжирования можно использовать NDCG
    'boosting_type': 'gbdt',
    'learning_rate': 0.05,
    'num_leaves': 31,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
}

# Обучение модели
model = lgb.train(params, lgb_train, valid_sets=[lgb_train, lgb_val], num_boost_round=1000)

# Предсказание на валидации для оценки
y_pred_val = model.predict(X_val, num_iteration=model.best_iteration)

from sklearn.metrics import mean_squared_error
rmse_val = mean_squared_error(y_val, y_pred_val, squared=False)
print(f"RMSE на валидационных данных: {rmse_val}")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.022292 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6119
[LightGBM] [Info] Number of data points in the train set: 857909, number of used features: 25
[LightGBM] [Info] Start training from score 3.580388
RMSE на валидационных данных: 0.9366604572113997


/Users/akzholba/Documents/RecSys/RecSysCompetition/.venv/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [7]:
import itertools
top_k = 10
def get_top_k_recommendations(user_id, k=top_k, df=test_df):
    user_predictions = df[df['user_id'] == user_id]
    top_k_items = user_predictions.nlargest(k, 'predicted_rating')['item_id'].values
    return list(top_k_items)
users = test_df['user_id'].unique()
items = test_df['item_id'].unique()

all_combinations_test = pd.DataFrame(list(itertools.product(users, items)), columns=['user_id', 'item_id'])
all_combinations_test = join_features_to_test(all_combinations_test, user_features_from_train, item_features_from_train)
# X_test, y_test = all_combinations_test.drop(target_col, axis = 1), all_combinations_test[target_col]
X_test = all_combinations_test
all_combinations_test['predicted_rating'] = model.predict(X_test, num_iteration=model.best_iteration)

# Для каждого пользователя выбираем топ-10 фильмов в тестовом наборе
user_top_k_recommendations_test = {user_id: get_top_k_recommendations(user_id, df=all_combinations_test) for user_id in test_df.user_id}

# Преобразуем результат в DataFrame для теста
recommendations_test = pd.DataFrame({
    'user_id': user_top_k_recommendations_test.keys(),
    'recommended_items': user_top_k_recommendations_test.values()
})

# Сохраняем результат для тестового набора

recommendations_test.to_csv('recommendations_lightGBM.csv', index=False)


In [13]:
recommendations_test = recommendations_test['recommended_items'].apply(lambda x: [str(a) for a in x]).apply(lambda x: ' '.join())

AttributeError: 'list' object has no attribute 'astype'

In [14]:
recommendations_test['recommended_items']

0       [2243, 3022, 1583, 841, 869, 36, 2564, 1543, 2...
1       [2243, 1543, 841, 3022, 2789, 1039, 2862, 2564...
2       [3022, 1543, 1583, 1039, 1831, 3529, 2862, 122...
3       [3022, 1583, 1831, 1039, 1543, 3529, 2862, 472...
4       [1543, 2243, 36, 1039, 2862, 841, 1560, 2789, ...
                              ...                        
6035    [2243, 841, 3022, 1543, 869, 1039, 2564, 36, 2...
6036    [2243, 841, 36, 2789, 869, 1543, 2365, 2564, 2...
6037    [2243, 841, 869, 36, 3618, 3653, 3072, 2365, 1...
6038    [2243, 1543, 841, 3022, 2862, 2564, 869, 3618,...
6039    [36, 2789, 2564, 1543, 2243, 779, 2862, 152, 2...
Name: recommended_items, Length: 6040, dtype: object